In [10]:
import polars as pl
from db.model import Tactics, Techniques, TacTech, Datasources, TechData, DataCollection, Cyberkillchain, AdditionalDatasources, Mitigations, TechMitigations
from miter.create_db import create_tactic_as_records, create_technique_as_records, create_mitigation_as_records

In [14]:
tactic_path = "/home/work/data/enterprise-attack-v15.1-tactics.xlsx"
technique_path = "/home/work/data/enterprise-attack-v15.1-techniques.xlsx"
mitigation_path = "/home/work/data/enterprise-attack-v15.1-mitigations.xlsx"
relation_path = "/home/work/data/enterprise-attack-v15.1-relationships.xlsx"

In [11]:
target_model = TechMitigations

In [12]:
target_model.drop_table()
target_model.create_table()

In [22]:
mitigation_df = pl.read_excel(relation_path)

In [23]:
map_replace_map = {"source ID": "mitigation_id", "target ID": 'technique_id'}

In [35]:
relation_df = pl.read_excel(relation_path)
relation_df = relation_df.filter(pl.col("source ID").str.starts_with("M"))
relation_df = relation_df.select(list(map_replace_map.keys())).rename(map_replace_map)
relation_df.with_columns((pl.arange(1, relation_df.height + 1)).alias("id"))
data = relation_df.to_dicts()

mitigation_id,technique_id,id
str,str,i64
"""M1036""","""T1110""",1
"""M1036""","""T1078.004""",2
"""M1036""","""T1110.004""",3
"""M1036""","""T1621""",4
"""M1036""","""T1110.001""",5
…,…,…
"""M1016""","""T1195.001""",1291
"""M1016""","""T1195.002""",1292
"""M1016""","""T1190""",1293


In [36]:
target_model.insert_many(data).execute()

1